#### Add additional metrics and scores.

In [1]:
import pandas as pd 
from google.cloud import bigquery

In [2]:
bq_conn = bigquery.Client()

In [3]:
sql_combine = """
CREATE OR REPLACE TABLE DS_WORK.PHYSICIAN_ORG_AFFILIATIONS
AS
SELECT A.*
  , CASE WHEN B.PHYSICIAN_NPI IS NOT NULL THEN 1 ELSE 0 END AS OTHER_SOURCE_FLAG
  , CASE WHEN CURRENT_DECAY_REMAINING_VAL >= 0.1 THEN 1 ELSE 0 END AS CURRENT_FLAG 
  , SCORE
  , SCORE_BUCKET 
  , CURRENT_DECAY_REMAINING_VAL
  , PCT_CLAIMS_MCR
  , PCT_CLAIMS_COMM
  , PCT_CLAIMS
  , ROW_NUMBER() OVER(PARTITION BY A.PHYSICIAN_NPI, A.FIRM_TYPE ORDER BY SCORE DESC) AS FIRM_TYPE_RANK 
  , PHYSICIAN_TOTAL_PATIENTS_COMM AS PATIENTS_COMM
  , PHYSICIAN_TOTAL_PATIENTS_MCR AS PATIENTS_MCR 
FROM   DS_WORK.PHYSICIAN_ORG_AFFILIATIONS A 
LEFT JOIN   (
              SELECT PHYSICIAN_NPI, DEFHC_ID 
              FROM (
                   SELECT NPI AS PHYSICIAN_NPI, HOSPITAL_ID AS DEFHC_ID FROM DS_WORK.CURRENT_PHYSICIAN_PG_AFFILIATIONS
                   UNION DISTINCT
                   SELECT NPI AS PHYSICIAN_NPI, HOSPITAL_ID AS DEFHC_ID FROM DS_WORK.CURRENT_PHYSICIAN_HOSPITAL_AFFILIATIONS
                   ) 
             ) B
ON   A.PHYSICIAN_NPI = B.PHYSICIAN_NPI 
AND  A.DEFHC_ID = B.DEFHC_ID 
LEFT JOIN DS_WORK.PHYSICIAN_ORG_AFFILIATION_DECAY C
ON   A.PHYSICIAN_NPI = C.PHYSICIAN_NPI 
AND  A.DEFHC_ID = C.DEFHC_ID 
LEFT JOIN DS_WORK.PHYSICIAN_ORG_AFFILIATION_SCORES D
ON   A.PHYSICIAN_NPI = D.PHYSICIAN_NPI 
AND  A.DEFHC_ID = D.DEFHC_ID
LEFT JOIN DS_WORK.PHYSICIAN_ORG_AFFILIATIONS_CLAIMS_PCT E
ON   A.PHYSICIAN_NPI = E.PHYSICIAN_NPI
AND  A.DEFHC_ID = E.DEFHC_ID 
LEFT JOIN DS_WORK.PHYSICIAN_AFFILIATION_TOTAL_PATIENTS_COMMERCIAL F
ON   A.PHYSICIAN_NPI = F.PHYSICIAN_NPI 
AND  A.DEFHC_ID = F.DEFHC_ID
LEFT JOIN DS_WORK.PHYSICIAN_AFFILIATION_TOTAL_PATIENTS_MEDICARE G
ON   CAST(A.PHYSICIAN_NPI AS STRING) = G.PHYSICIAN_NPI 
AND  A.DEFHC_ID = G.DEFHC_ID
"""

bq_conn.query(sql_combine).result()